# Read in the dataset

In [1]:
import gzip
import os
import humanfriendly
import numpy as np
import pandas as pd

from openclean.data.source.socrata import Socrata

import warnings
warnings.filterwarnings('ignore')

# Set the unique id of the dataset you want to use here
db_id = 'bty7-2jhb'

dataset = Socrata().dataset(db_id)

datafile = f'./{db_id}.tsv.gz'

# Download file if it doesn't exist
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as  f:
        print('Downloading ...\n')
        dataset.write(f)

fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print(f'Using "{dataset.name}" in file {datafile} of size {fsize}')

Using "Historical DOB Permit Issuance" in file ./bty7-2jhb.tsv.gz of size 321.34 MB


## Create data stream

In [2]:
from openclean.pipeline import stream

ds_full = stream(datafile)

## Clean city data

In [3]:
from fuzzywuzzy import fuzz
import json

from openclean.data.refdata import RefStore
from openclean.function.value.null import is_empty

def clean_city_name(name, valid_city_lookup):
    if is_empty(name):
        return 'N/A'
    return valid_city_lookup.get(name.upper(), 'N/A')


def clean_city_data(ds, column_name):
    # Download the license plate state codes dataset.
    refdata = RefStore()
    refdata\
        .load('encyclopaedia_britannica:us_cities', auto_download=True)\
        .df()\

    # Get set of distinct state codes.
    city_ref = refdata.load('encyclopaedia_britannica:us_cities', auto_download=True).distinct('city')
    # Get list of distinct owner house city names
    city_names = ds.distinct(column_name)
    # Init lookup dictionary for fuzzy matching
    city_ref_lookup = {}
    ref_file_name = 'city_ref_lookup.json'
    # 
    if os.path.isfile(ref_file_name):
        with open(ref_file_name) as f:
            city_ref_lookup = json.load(f)
    # compare city name to each valid city and add to lookup table
    # if the similarity is high enough
    for city in city_names:
        if is_empty(city):
            continue
        name = city.upper()
        if city_ref_lookup.get(name):
            continue
        if (name == 'NYC' or 
              name == 'NY' or
              fuzz.ratio('NY', name) > 70 or
              fuzz.ratio('NYC', name) > 70):
            city_ref_lookup[name] = 'NEW YORK'
            continue
        found_match = False
        for valid_city in city_ref:
            percent_match = fuzz.ratio(valid_city.upper(), name)
            if percent_match > 70:
                found_match = True
                city_ref_lookup[name] = valid_city.upper()
                break
        if not found_match:
            city_ref_lookup[name] = 'N/A'

    cleaned_data = ds.update(column_name, lambda x: clean_city_name(x, city_ref_lookup))
    with open(ref_file_name, 'w') as f:
        json.dump(city_ref_lookup, f)
    return cleaned_data

# Second argument is column name in dataset for city data
clean_city_data(ds_full, 'Owner’s House City')